In [1]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import mysql.connector
import pickle

import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [2]:
# Connect to MySQL database
conn = mysql.connector.connect(host='127.0.0.1', user='root', passwd=password)
cursor = conn.cursor()

# Create or select the database
cursor.execute("CREATE DATABASE IF NOT EXISTS vanguard")
cursor.execute("USE vanguard")

In [4]:
def pickle_to_mysql_table(cursor, pickle_file, table_name):
    # Read the interactions from the pickle file
    with open(pickle_file, 'rb') as file:
        interactions = pickle.load(file)

    # Define MySQL table schema based on DataFrame columns
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in interactions.columns])})"
    cursor.execute(create_table_query)

    # Convert DataFrame to list of dictionaries, handling NaN values
    data = interactions.where((pd.notnull(interactions)), None).to_dict(orient='records')

    # Insert data into MySQL table
    for row in data:
        columns = ', '.join(row.keys())
        values = ', '.join(['%s'] * len(row))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"
        cursor.execute(insert_query, list(row.values()))

In [ ]:
pickle_to_mysql_table(cursor, clients.pkl, clients)
pickle_to_mysql_table(cursor, group_id.pkl, group_id)
pickle_to_mysql_table(cursor, interactions.pkl, interactions)